In [1]:
import os
import sys
import nbimporter

# Ensure current directory is on sys.path
cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.append(cwd)
    
from template_manager import template_selector
from patient_data_extraction import extract_patient_info,image_extractor,generate_motility_report,get_measure_table,get_measurements,get_mot_table,mot_extractor
from file_conversion import cargar_path
from aux_calculations import remove_signs,expand_dict_with_lists_inplace, convert_to_int,calc_e_e_stress,expand_dict_with_lists_inplace
from docx import Document
from docxtpl import DocxTemplate



def render_template(path)-> None:
    '''
    acepta un path y determina si es cardiaco o stress, si son estos extrae medidas, tratornos de motilidad y mapa polar
    Si no lo es, extrae solamente datos del paciente y las imagenes del archivo docx producido por el equipo vinno
    guarda la renderizaci'on en la misma carpeta donde estaba el docx para poder realizar una comparacion
        
    '''
    doc=Document(path)
    template,tipo=template_selector(path)
    info_pac= extract_patient_info(doc)
    image=image_extractor(doc,template,tipo=tipo)
    #guardo el save path con nombre tipo de estudio y fecha en la misma carpeta.
    save_path=str(path).rsplit('\\',1)[0]+f'\\{info_pac["Name"]} {tipo} {info_pac["Exam_Date"]}.docx'
    print(save_path)
    print(tipo)
    #Si es carotida solo van los datos del paciente
    if tipo in ['card','stress']: 
        measurements_table=get_measure_table(doc)
        measurements_dic=get_measurements(measurements_table,info_pac['Gender']) 
        if tipo =='stress':
            mot_table=get_mot_table(doc)
            mot=mot_extractor(mot_table)
            mot_report=generate_motility_report(mot)
            expand_dict_with_lists_inplace(measurements_dic) #preparo el dict para la fx calc_e_e_stress
            measurements_dic['E_e_rel'],measurements_dic['e_e_avg']=calc_e_e_stress(measurements_dic)

            context={**info_pac,**measurements_dic,'image': image['image'], 'mot': mot['mot'],**mot_report}
        
        else:
            context={**info_pac,**measurements_dic,'image': image['image']}
        
    else: 
        context={**info_pac,'image': image['image']}
    template.render(context)
    template.save(save_path)
    print(f'informe {info_pac["Name"]} creado con exito')

In [2]:
lista_de_trabajo=cargar_path(r"C:\Users\lucho\Documents\temp\documentos")


Converted C:\Users\lucho\Documents\temp\documentos\[albertoacherbi]-[20250528009]-[694aea1e-b8f3-473e-a55d-e72efaee0205]\18_F4-12L_Carotid.Report.V3.doc to C:\Users\lucho\Documents\temp\documentos\[albertoacherbi]-[20250528009]-[694aea1e-b8f3-473e-a55d-e72efaee0205]\18_F4-12L_Carotid.Report.V3.docx
Converted C:\Users\lucho\Documents\temp\documentos\[albertoacherbi]-[20250528009]-[694aea1e-b8f3-473e-a55d-e72efaee0205]\19_F4-12L_Carotid.Report.V3.doc to C:\Users\lucho\Documents\temp\documentos\[albertoacherbi]-[20250528009]-[694aea1e-b8f3-473e-a55d-e72efaee0205]\19_F4-12L_Carotid.Report.V3.docx
Converted C:\Users\lucho\Documents\temp\documentos\[albertoacherbi]-[20250528009]-[694aea1e-b8f3-473e-a55d-e72efaee0205]\20_S1-6P_card2.Report.V3.doc to C:\Users\lucho\Documents\temp\documentos\[albertoacherbi]-[20250528009]-[694aea1e-b8f3-473e-a55d-e72efaee0205]\20_S1-6P_card2.Report.V3.docx
Converted C:\Users\lucho\Documents\temp\documentos\[albertoportella]-[20250609005]-[356a93f7-737f-4a43-b63

In [4]:
for index,i in enumerate(lista_de_trabajo):
    print(f'archivo N {index}')
    render_template(i)

# error '>=' not supported between instances of 'list' and 'int' antoniorocco


archivo N 6
C:\Users\lucho\Documents\temp\documentos\[estherramirez]-[20250609007]-[19411236-cd05-46c1-b933-329711ac122d]\ramirez gregoria esther carotid 06-09-2025.docx
carotid
informe ramirez gregoria esther creado con exito
archivo N 7
C:\Users\lucho\Documents\temp\documentos\[estherramirez]-[20250609007]-[19411236-cd05-46c1-b933-329711ac122d]\ramirez gregoria esther carotid 06-09-2025.docx
carotid
informe ramirez gregoria esther creado con exito
archivo N 8
C:\Users\lucho\Documents\temp\documentos\[estherramirez]-[20250609007]-[19411236-cd05-46c1-b933-329711ac122d]\ramirez gregoria esther card 06-09-2025.docx
card
informe ramirez gregoria esther creado con exito
archivo N 9
C:\Users\lucho\Documents\temp\documentos\[estherramirez]-[20250609007]-[19411236-cd05-46c1-b933-329711ac122d]\ramirez gregoria esther card 06-09-2025.docx
card
informe ramirez gregoria esther creado con exito
archivo N 10
C:\Users\lucho\Documents\temp\documentos\[garbielelfren]-[20250609009]-[d288992d-a28a-4491-

In [6]:
from pathlib import Path
path=r"C:\Users\lucho\Documents\temp\documentos\[anthonycollins]-[20250515004]-[c4d44b2c-be4e-4c15-9bb3-59444c7a2122]\12_S1-6P_card2.Report.V3.docx"
path=Path(path)
render_template(path)

C:\Users\lucho\Documents\temp\documentos\[anthonycollins]-[20250515004]-[c4d44b2c-be4e-4c15-9bb3-59444c7a2122]\collins anthony card 05-15-2025.docx
card
informe collins anthony creado con exito


# Context dict:

## Patient data
    {
    'Name': str, 'Patient_ID': str, 
    'Age': str(edad)+ 'y', 'Gender': str, 
    'Exam_Date': str(mm-dd-yyyy), 
    'Perf.Physician': str, 
    'Ref.Physician': str, 
    'Operator': str
    }
    
## Measurements (if more than one, list of the same type)
    {
    'CARD_card2_2D_General': None, 'IVC_Diam': mm float or list float, 
    'IVSd': mm int, 'LVIDd': mm int , 'LVPWd': mm int, 'IVSs': mm float, 'LVIDs': mm int, 'LVPWs': mm float,
    'Ao_Diam': mm int, 'RAAd': diam cm2 int, 'Distance': mm int, 'LAAd': volume ml/m2 float,
    'A4C_Simpson': % str, 'LVEDV': float ml, 'LVESV': float ml, 'LVAd': float cm2, 'LVLd': float cm2, 'LVAs': float,   'LVLs':float, 
    'CARD_card2_M_General': None, 'TAPSE': float mm, 
    'CARD_card2_Doppler_General': None, 
    'AV_Vmax': float m/seg, 'AV_Vmax_grad': float mmHg, 'PG': float mmHg, 
    'LVOT_Vmax': float m/seg, 'LVOT_Vmax_grad': float mmHg, 
    'MV_Vel_E': float m/seg, 'MV_Vel_E_grad': float mmHg, 'MV_Vel_A': float m/seg, 'MV_Vel_A_grad': float mmHg,
    'Med_Vel_E': float cm/seg, 'Lat_Vel_E': float cm/seg, 'IVRT': float msec,
    'Calculation_Item': 'Result', 'card2': None, 'FS_2D': % int, 'LVIDs_Index_2D': ml/m2 float, 'LVd_Mass_2D_ASE': float gr,
    'LVd_Mass_Index_2D_ASE': int gr/m2, 'RWT_2D': dimless %, 'IVS_2D': 7.16, 'LVPW_2D': 1.12, 'LVESVI_A4C_Simp': float ml/m2, 'CSA_AV_SV': 7.61, 'MV_E_A_Ratio': float, 'AV_VR': float, 'E_Med_E': float, 'E_Lat_E': float, 'Average_E': float, 'E_Avg_E': float
    }
    
## Image

    { 'key': 'imageN',
      'image': docxtpl.inline_image.InlineImage,
      'key': 'imageN+1',
      'image':docxtpl.inline_image.InlineImage,
      ...
    }
    
## MOT

    {
    
    }
 

# funcion para el GUI

In [ ]:
import nbimporter
from template_manager import template_selector_gui

def render_template_gui(study_path,template_path)-> None:
    '''
    acepta un path y determina si es cardiaco o stress, si son estos extrae medidas, tratornos de motilidad y mapa polar
    Si no lo es, extrae solamente datos del paciente y las imagenes del archivo docx producido por el equipo vinno
    guarda la renderizaci'on en la misma carpeta donde estaba el docx para poder realizar una comparacion
        
    '''
    doc=Document(study_path)
    template,tipo=template_selector_gui(template_path,study_path)
    info_pac= extract_patient_info(doc)
    image=image_extractor(doc,template,tipo=tipo)
    #guardo el save path con nombre tipo de estudio y fecha en la misma carpeta.
    save_path=str(study_path).rsplit('\\',1)[0]+f'\\{info_pac["Name"]} {tipo} {info_pac["Exam_Date"]}.docx'
    #Si es carotida solo van los datos del paciente
    if tipo in ['card','stress']: 
        measurements_table=get_measure_table(doc)
        measurements_dic=get_measurements(measurements_table,info_pac['Gender'])
        if tipo =='stress':
            mot_table=get_mot_table(doc)
            mot=mot_extractor(mot_table)
            mot_report=generate_motility_report(mot)
            expand_dict_with_lists_inplace(measurements_dic) #preparo el dict para la fx calc_e_e_stress
            measurements_dic['E_e_rel'],measurements_dic['e_e_avg']=calc_e_e_stress(measurements_dic)

            context={**info_pac,**measurements_dic,'image': image['image'], 'mot': mot['mot'],**mot_report}
        
        else:
            context={**info_pac,**measurements_dic,'image': image['image']}
        
    else: 
        context={**info_pac,'image': image['image']}
    template.render(context)
    template.save(save_path)
    print(f'informe {info_pac["Name"]} creado con exito')

In [ ]:
study_path=r"C:\Users\lucho\Documents\temp\documentos\[alcideslescano]-[20241230004]-[e4eda042-3275-48e6-ad59-bbf3de8797e2]\16_S1-6P_card2.Report.V3.docx"
template_path=r'C:\Users\lucho\Desktop\Plantillas'
render_template_gui(study_path,template_path)